In [1]:
#Imports
from IPython.display import display, Math
import sympy as sp
from scs.utils_opt import *
from swg.sw_procedure import Hamiltonian
from scs.supercoeffs import Circuit, Cnlp

# Kerr-cat optimization

<span style="font-size:18px;  font-weight:plain;"> 1. Formulate the concrete problem involving parametric drives, and move into a rotating frame   $\hat{U}_R=\exp{(i\omega^\prime\hat{a}^\dagger\hat{a}t)}$, $\hat{U}_{R}^{\dagger}\hat H\hat{U}_{R}-i\hat{U}_{R}^\dagger\dot{\hat{U}}_{R}\rightarrow \hat H$, where $\omega^\prime\approx\frac{q}{r}\omega_d$,  $q,\,r\in \mathcal{Z}$, and  $\omega_0-\omega^\prime\ll\omega_0$, so 
    \begin{align}   
        % \begin{split}
            \hat{H}(t)=(\omega_0-\omega^\prime)\hat{a}^{\dagger}\hat{a}+\,\sum\limits_{\substack{n=0\\ l=0\\p=0}}^{\{l,p\}^\prime}{C}_{nl,p}\left(\hat{a}^{\dagger n}\hat{a}^{n+l}\right.&\left.e^{-il\omega^\prime{t}}(e^{ip{\omega}_d{t}}+\text{c.c.})+\text{H.c.}\right),\nonumber
        % \end{split}
    \end{align}
where we take $\gamma=0$ for the sake of brevity.
At this point, depending on the type of wave mixing involved, one chooses the order up to which the Hamiltonian expansion is taken. In the main text, we limit the expansion with $2n+l+p\leq4$ and $p\leq2$ for a Kerr parametric oscillator dealing with three- and four-wave mixing as the dominant processes and with $2n+l+p\leq4$ for the beam-splitter interaction problem involving additionally five-wave mixing. Such an approximation, in particular, is justified by the smallness of $\varphi_\mathrm{zpf}$ and $C_{nl,p}\propto\varphi_\mathrm{zpf}^{2n+l}$.</span>

In [2]:
sw_order = 2      # Schrieffer-Wolff procedure order (RWA is zero order)
max_order = 4     # maximum order of terms to consider in normal-ordered time-dependent 
                  #Hamiltonian 2n+l+p ≤ 4 (p set to be ≤ 3)
dr_mode_dim = 1   # number of drives in the circuit
hamiltonian = Hamiltonian('Kerr-cat', dr_mode_dim, sw_order=sw_order, max_order=max_order)
hamiltonian.add_freqs_condition([2],[1])# the condition 2*\omega^prime=omega_d is ensured 

## Schrieffer-Wolff procedure for single DOF circuit
<span style="font-size:17px;  font-weight:plain;"> 2. Calculate the static effective Hamiltonian up to a required order using the Schrieffer-Wolff (SW) procedure or any alternative averaging method. In the main text, we employ the procedure introduced in 10.1103/PhysRevLett.129.100601 computing the unitary generator $\hat S(t)$ for the canonical transformation and  obtaining a static effective Hamiltonian
via 
\begin{align}\label{eq_app:effH}
\begin{split}
\hat{\mathcal{H}}_{\mathrm{eff}} \equiv \overline{e^{\hat{S} / i \hbar} \hat{H}(t) e^{-\hat{S} / i \hbar}}-i \hbar\overline{ e^{\hat{S} / i \hbar} \partial_{t} e^{-\hat{S} / i \hbar}}\\
= \overline{\hat{{H}}(t)}+ \frac{1}{i\hbar}\overline{[\hat S, \hat{{H}}(t)]}+\frac{1}{2!(i\hbar)^2}\overline{[\hat S,[\hat S, \hat{{H}}(t)]]} \\+ \cdots  + \overline{\partial_t\hat S}  + \frac{1}{2! i\hbar} \overline{[\hat S, \partial_t {\hat S}]} + \cdots
\end{split}
\end{align}
where the  Baker-Campbell-Hausdorff formula\footnote{$e^{-\hat A} \hat B e^{\hat A} = \hat B + [\hat B, \hat A] + \frac{1}{2!} [[\hat B, \hat A],\hat A] + \ldots$} was applied and the time averaging is given by $\overline{f}(t)\equiv\frac{1}{T}\int_0^T{f(t)}dt$, where $T=2\pi/\omega$. Then, both the generator $\hat S$ and the effective Hamiltonian $\hat{\mathcal{H}}_{\mathrm{eff}}$ can be calculated perturbatively with a recursive procedure up to the required order of $1/\omega$ considering the expansion of the time-dependent Hamiltonian 
\begin{equation}
\hat{{H}}(t)=\sum\limits_{m\in \mathbb{Z}}\hat{H}_me^{im\omega t},
\end{equation}
so 
\begin{align}    \hat{\mathcal{H}}_{\mathrm{eff}} = \sum_{n\in\mathbb{N}} \hat{\mathcal{H}}_{\mathrm{eff}}^{(n)}, \quad \hat S = \sum_{n\in\mathbb{N}} \hat S^{(n)}.
\end{align}
The RWA for the Hamiltonian is defined as $\hat{\mathcal{H}}_{\mathrm{eff}}^{(0)}=\overline{\hat{H}(t)}$ with $\hat S^{(0)}=0$, while the first-order correction is obtained from
\begin{align}
\frac{\hat S^{(1)}}{\hbar}&=- \int dt\; \textbf{osc}\hat{H}(t),\\
\hat{\mathcal{H}}_{\mathrm{eff}}^{(1)}&=\frac{1}{i\hbar}\overline{\left([\hat S^{(1)}, \hat{{H}}(t)]+\frac{1}{2!} [\hat S^{(1)}, \partial_t {\hat S^{(1)}}]\right)},
\end{align}
where $\textbf{osc}f= f-\overline{f}$. The next order of perturbations can be calculated via the recursive procedure presented in the original paper 10.1103/PhysRevLett.129.100601. Although it is possible to obtain corrections up to the required order, for simplicity, the main text and next sections focus only on the first-order correction beyond the RWA.</span>

In [3]:
# Monimial to calculate effective parametric amplitudes for in format (a^+)^k1a^k2 --> ((k1,), (k2,))
FINAL_MONOMS = {
    "DETUNING":          ((1,),(1,)),   # (a^+)a
    "KERR":              ((2,),(2,)),   # (a^+)^2a^2
    "2PH_SQUEEZING":     ((2,),(0,))    # (a^+)^2
}
#Performs Schrieffer-Wolff procedure for the defined model and saves the result to the file.
#In current version, it needs to be recalculated for the same model if parameters of SW procedure or Hamiltonian have changed.
model = hamiltonian.build_model(FINAL_MONOMS.values(), recalculate=True)
omegad = sp.symbols('omega_d', latex_name='\\omega_d')
omega0 = omegad/2

#Show 0th order Schrieffer-Wolff corrections
kerr,_ = effective_hamiltonian_coeff_to_latex(model[FINAL_MONOMS["KERR"]][0], omega0, omegad)
delta,_ = effective_hamiltonian_coeff_to_latex(model[FINAL_MONOMS["DETUNING"]][0], omega0, omegad)
eps2,_ = effective_hamiltonian_coeff_to_latex(model[FINAL_MONOMS["2PH_SQUEEZING"]][0], omega0, omegad)
print("Self-Kerr amplitude, 0th order:")
display(Math(kerr))
print("Detuning amplitude, 0th order:")
display(Math(delta))
print("Two-photon squeezing amplitude, 0th order:")
display(Math(eps2))

#Show 1st order Schrieffer-Wolff corrections
kerr,_ = effective_hamiltonian_coeff_to_latex(model[FINAL_MONOMS["KERR"]][1], omega0, omegad)
delta,_ = effective_hamiltonian_coeff_to_latex(model[FINAL_MONOMS["DETUNING"]][1], omega0, omegad)
eps2,_ = effective_hamiltonian_coeff_to_latex(model[FINAL_MONOMS["2PH_SQUEEZING"]][1], omega0, omegad)
print("Self-Kerr amplitude, 1st order:")
display(Math(kerr))
print("Detuning amplitude, 1st order:")
display(Math(delta))
print("Two-photon squeezing amplitude, 1st order:")
display(Math(eps2))

#Show 2st order Schrieffer-Wolff corrections
kerr,_ = effective_hamiltonian_coeff_to_latex(model[FINAL_MONOMS["KERR"]][2], omega0, omegad)
delta,_ = effective_hamiltonian_coeff_to_latex(model[FINAL_MONOMS["DETUNING"]][2], omega0, omegad)
eps2,_ = effective_hamiltonian_coeff_to_latex(model[FINAL_MONOMS["2PH_SQUEEZING"]][2], omega0, omegad)
print("Self-Kerr amplitude, 2nd order:")
display(Math(kerr))
print("Detuning amplitude, 2nd order:")
display(Math(delta))
print("Two-photon squeezing amplitude, 2nd order:")
display(Math(eps2))

((1,), (1,))
((2,), (2,))
((2,), (0,))
Initialized and saved array for model 'Kerr-cat' to file.
Self-Kerr amplitude, 0th order:


<IPython.core.display.Math object>

Detuning amplitude, 0th order:


<IPython.core.display.Math object>

Two-photon squeezing amplitude, 0th order:


...

<IPython.core.display.Math object>

Detuning amplitude, 2nd order:


<IPython.core.display.Math object>

Two-photon squeezing amplitude, 2nd order:


<IPython.core.display.Math object>

## Parametric amplitudes for the effective Hamiltonian 

<span style="font-size:20px;  font-weight:plain;"> 3. Obtain the supercoefficients for circuit designs of interest by using  Eqs. (B4), (B5) for symmetric Josephson circuits or Eq.\  (A12}) for circuits with the geometric inductance or broken permutation symmetry. This is the first step where the specific potential function must be considered. In the main text, we describe circuits consisting of an array of SNAILs or asymmetric SQUIDs (see Table I in the main text).</span>

<span style="font-size:20px;  font-weight:plain;">4. Define a list of free parameters to consider (in the main text, for example, $\varphi_e$, $N$, $M$, $\alpha_\mathrm{s}$, $x_J$) and fixed parameters ($E_C$, $E_J$). In the main text, we employ an effective drive amplitude $\Pi=n_\mathrm{zpf}\tilde{\Pi}$ often used to describe the input from the capacitive parametric drive \cite{frattini2022squeezed,ChapmanStijn2023,jaya2022lind}. For flux-driven circuits, we use $\tilde{\Pi}\equiv\Pi_a-\Pi_b=2\varphi_\mathrm{ac0}M$ (see details above).</span>

In [4]:
#Parameters for the circuit topology
M=2      
N=3
xJ = 0.3 # Current implementation considers cirtuit with xJ>500 as having no parasitic inductance and closed form for SCs
alpha = 0.3
phi_ext = 0.33
energy_c_large = 0.12 # GHz 
energy_jj = 400 # GHz
config= 0  # Array of SNAILS for config= 0 and Array of SQUIDS for config= 1
maxorder = 8 # -1 for circuits without parasitic inductance
Pi = 0.5 # Drive amplitude $\Pi=n_\mathrm{zpf}\Tilde{\Pi}$, $\Tilde{\Pi} = \Omega_d{\omega}_d/(\omega_d^2{-}\omega_0^2)$ ,see Eq. (2), (6)

circuit = Circuit(M, xJ, N, alpha, phi_ext, energy_jj, energy_c_large, config, maxorder=maxorder)

omega_d = calculate_omega_d(circuit, Pi)
print("Cnlp for n = 0, l = 2, p = 1 (0th order two-photon squeezing):", Cnlp(circuit, Pi, 0, 2, 1))
kerr_1st_order = effective_hamiltonian_coeff([model[FINAL_MONOMS["KERR"]][0]], circuit, Pi, omega_d/2, omega_d)
print("First-order correction for the self-Kerr:", kerr_1st_order)
kerr_012_orders = effective_hamiltonian_coeff(model[FINAL_MONOMS["KERR"]], circuit, Pi, omega_d/2, omega_d)
print("Self-Kerr all SW orders,","sw_order =", sw_order, ", calculated for the model:", kerr_012_orders)

Cnlp for n = 0, l = 2, p = 1 (0th order two-photon squeezing): -0.04406228545153307
First-order correction for the self-Kerr: -0.0023196264205310718
Self-Kerr all SW orders, sw_order = 2 , calculated for the model: -0.0034017374886158697


<span style="font-size:20px;  font-weight:plain;"> 5. Define the limitations of a simulation. This step is highly problem-specific and is primarily determined by the intended utility of the parametric processes. For the Kerr-cat problem, we introduce the minimal value of Kerr nonlinearity considered. For the beam-splitter interaction problem, we ensure the validity of the dispersive approximation and introduce the minimal value of cavity-cavity cross-Kerr, $\chi_{ab}$, considered (see next subsection). So, for example, to find the maximum  Kerr-cat size, for each circuit design we fix parameters $N$, $M$, $\alpha_\mathrm{s}$, $x_J$, $\Pi$, require $|K({\Pi})|>K_\mathrm{lim}=1\,\mathrm{MHz}$ and scan all values of external flux, $\varphi_e$.</span>

In [5]:
Klim = 1e-3 # 1 MHz
max_drive_amp = 0.5 # Maximum drive amplitude to consider

## Sweeping flux bias to find maximum size of the Kerr-cat
<span style="font-size:17px;  font-weight:plain;"> Softly checks for maximum of Kerr-cat size by sweeping flux bias and looking for driven Kerr-free flux bias.
If there is self-Kerr K=0 estimates the corresponding flux bias 
which futher used for fine-tuning to point with K=>Klim </span>

In [6]:
load_or_initialize_kerr_cat_model() # makes the Kerr-cat model available internally for the Kerr-cat specific methods
x, lazy_phi_e_max = lazy_size_max(circuit, max_drive_amp = max_drive_amp)
print(lazy_phi_e_max)

Loaded array for model 'Kerr-cat' from file.
0.4947382695520874


<span style="font-size:17px;  font-weight:plain;"> Returns maximum Kerr-cat size for minimal self-Kerr magnitude for particular circuit scheme by looking around K=0 flux bias</span>

In [7]:
max_size, phi_e_max = maxSize_opt(circuit, lazy_phi_e_max, Klim = Klim, max_drive_amp = max_drive_amp)
print('Max size and its phi_ext:', max_size, phi_e_max)

Max size and its phi_ext: 65.34491596550693 0.4946232695520873


<span style="font-size:17px;  font-weight:plain;"> For a different drive amplitude:</span>

In [8]:
max_drive_amp = 1. # Maximum drive amplitude to consider
_, lazy_phi_e_max = lazy_size_max(circuit, max_drive_amp = max_drive_amp)
print(lazy_phi_e_max)

0.4945976246085904


In [9]:
max_size, phi_e_max = maxSize_opt(circuit, lazy_phi_e_max, Klim = Klim, max_drive_amp = max_drive_amp)
print('Max size and its phi_ext:', max_size, phi_e_max)

Max size and its phi_ext: 113.63318073637957 0.4947326246085905
